In [11]:
import pandas as pd
from datetime import datetime
from scipy.optimize import newton
import yfinance as yf

In [2]:
# 1. Load your Firstrade CSV export
df = pd.read_csv('./historical_data/FT_CSV_90163986.csv')

# 2. Filter for executed trades
df_trades = df[df['RecordType'] == 'Trade'].copy()
df_trades['TradeDate'] = pd.to_datetime(df_trades['TradeDate'], errors='coerce')

In [3]:
df_trades

,Symbol,Quantity,Price,Action,Description,TradeDate,SettledDate,Interest,Amount,Commission,Fee,CUSIP,RecordType
3,VOO,0.97671,511.9200,BUY,VANGUARD S&P 500 ETF ...,2024-07-17,2024-07-18,0.0,-500.00,0.0,0.0,922908363,Trade
4,ASML,0.31798,943.4299,BUY,***ASML HOLDING N V N Y REGISTRY SH...,2024-07-17,2024-07-18,0.0,-299.99,0.0,0.0,N07059210,Trade
6,NVDA,1.27866,117.3099,BUY,NVIDIA CORP ...,2024-07-18,2024-07-19,0.0,-150.00,0.0,0.0,67066G104,Trade
7,META,0.42847,466.7700,BUY,META PLATFORMS INC CLASS A COMMON ...,2024-07-30,2024-07-31,0.0,-200.00,0.0,0.0,30303M102,Trade
8,VOO,1.39439,502.0100,BUY,VANGUARD S&P 500 ETF ...,2024-07-30,2024-07-31,0.0,-700.00,0.0,0.0,922908363,Trade
...,...,...,...,...,...,...,...,...,...,...,...,...,...
466,NVDA,5.20074,96.1400,BUY,NVIDIA CORP ...,2025-04-21,2025-04-22,0.0,-500.00,0.0,0.0,67066G104,Trade
467,CMG,17.23859,46.4075,BUY,CHIPOTLE MEXICAN GRILL INC COMMON STOCK ...,2025-04-22,2025-04-23,0.0,-800.00,0.0,0.0,169656105,Trade
468,CAVA,5.43832,91.9400,BUY,CAVA GROUP INC COMMON STOCK ...,2025-05-06,2025-05-07,0.0,-500.00,0.0,0.0,148929102,Trade
470,SG,27.39350,18.2525,BUY,SWEETGREEN INC CLASS A COMMON ...,2025-05-06,2025-05-07,0.0,-500.00,0.0,0.0,87043Q108,Trade


In [4]:
# 3. Build a list of (date, cash_flow) tuples
#    Buys are negative amounts, sells positive
cashflows = list(zip(df_trades['TradeDate'], df_trades['Amount']))
cashflows.sort(key=lambda x: x[0])

In [13]:
cashflows

[(Timestamp('2024-07-17 00:00:00'), -500.0),
 (Timestamp('2024-07-17 00:00:00'), -299.99),
 (Timestamp('2024-07-18 00:00:00'), -150.0),
 (Timestamp('2024-07-30 00:00:00'), -200.0),
 (Timestamp('2024-07-30 00:00:00'), -700.0),
 (Timestamp('2024-07-30 00:00:00'), -100.0),
 (Timestamp('2024-07-30 00:00:00'), -100.0),
 (Timestamp('2024-08-01 00:00:00'), -75.0),
 (Timestamp('2024-08-01 00:00:00'), -75.0),
 (Timestamp('2024-08-01 00:00:00'), -50.0),
 (Timestamp('2024-08-05 00:00:00'), -50.0),
 (Timestamp('2024-08-05 00:00:00'), -50.0),
 (Timestamp('2024-08-06 00:00:00'), -49.99),
 (Timestamp('2024-08-06 00:00:00'), -50.0),
 (Timestamp('2024-08-06 00:00:00'), -100.0),
 (Timestamp('2024-08-06 00:00:00'), -49.99),
 (Timestamp('2024-08-06 00:00:00'), -100.0),
 (Timestamp('2024-08-06 00:00:00'), -50.0),
 (Timestamp('2024-08-07 00:00:00'), -50.0),
 (Timestamp('2024-08-07 00:00:00'), -100.0),
 (Timestamp('2024-08-07 00:00:00'), -100.0),
 (Timestamp('2024-08-07 00:00:00'), -49.99),
 (Timestamp('2024

### Append the current value of the total asset
You can get this from Firstrade.

In [18]:
cashflows

[(Timestamp('2024-07-17 00:00:00'), -500.0),
 (Timestamp('2024-07-17 00:00:00'), -299.99),
 (Timestamp('2024-07-18 00:00:00'), -150.0),
 (Timestamp('2024-07-30 00:00:00'), -200.0),
 (Timestamp('2024-07-30 00:00:00'), -700.0),
 (Timestamp('2024-07-30 00:00:00'), -100.0),
 (Timestamp('2024-07-30 00:00:00'), -100.0),
 (Timestamp('2024-08-01 00:00:00'), -75.0),
 (Timestamp('2024-08-01 00:00:00'), -75.0),
 (Timestamp('2024-08-01 00:00:00'), -50.0),
 (Timestamp('2024-08-05 00:00:00'), -50.0),
 (Timestamp('2024-08-05 00:00:00'), -50.0),
 (Timestamp('2024-08-06 00:00:00'), -49.99),
 (Timestamp('2024-08-06 00:00:00'), -50.0),
 (Timestamp('2024-08-06 00:00:00'), -100.0),
 (Timestamp('2024-08-06 00:00:00'), -49.99),
 (Timestamp('2024-08-06 00:00:00'), -100.0),
 (Timestamp('2024-08-06 00:00:00'), -50.0),
 (Timestamp('2024-08-07 00:00:00'), -50.0),
 (Timestamp('2024-08-07 00:00:00'), -100.0),
 (Timestamp('2024-08-07 00:00:00'), -100.0),
 (Timestamp('2024-08-07 00:00:00'), -49.99),
 (Timestamp('2024

In [16]:
today = datetime(2025, 5, 12)
V_now = 214,749.33
cashflows.append((today, +V_now))

TypeError: bad operand type for unary +: 'tuple'

In [5]:
# 4. Define XNPV and XIRR functions
def xnpv(rate, cashflows):
    t0 = cashflows[0][0]
    return sum(cf / ((1 + rate) ** ((date - t0).days / 365.0)) for date, cf in cashflows)

def xirr(cashflows):
    # Solve for rate that makes XNPV = 0
    try:
        return newton(lambda r: xnpv(r, cashflows), 0.1)
    except RuntimeError:
        return float('nan')

In [8]:
# 5. Compute your portfolio XIRR
portfolio_irr = xirr(cashflows)
print(f"Portfolio XIRR (annualized): {portfolio_irr:.2%}")

Portfolio XIRR (annualized): nan%


In [12]:
# --------------------------------------------------------------------
# Compare against S&P 500 benchmark
# --------------------------------------------------------------------
import yfinance as yf

start_date = cashflows[0][0].strftime('%Y-%m-%d')
end_date   = cashflows[-1][0].strftime('%Y-%m-%d')

sp = yf.download('^GSPC', start=start_date, end=end_date)
sp = sp['Adj Close']
sp_start = sp.iloc[0]
sp_end   = sp.iloc[-1]

years = (sp.index[-1] - sp.index[0]).days / 365.0
sp_annual_return = (sp_end / sp_start) ** (1 / years) - 1
print(f"S&P 500 annualized return: {sp_annual_return:.2%}")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


KeyError: 'Adj Close'

In [ ]:






# --------------------------------------------------------------------
# You can extend this script to:
# - Compute per-trade returns and compare each to the index over its holding period
# - Generate a time-weighted return (TWR) series
# - Visualize cash flows and cumulative performance with matplotlib
# --------------------------------------------------------------------
